<br>

## **Objective: To train CNN model and use trained model for prediction**

<br>

## **Step 1: Import**

In [1]:
from datetime import datetime
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as nnf
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [2]:
torch.set_printoptions(linewidth=300)

<br>

## **Step 2: Read and transform**

**Train dataset**

In [3]:
pf1 = pd.read_pickle('./data/cifar-10-batches-py/data_batch_1')
pf2 = pd.read_pickle('./data/cifar-10-batches-py/data_batch_2')
pf3 = pd.read_pickle('./data/cifar-10-batches-py/data_batch_3')
pf4 = pd.read_pickle('./data/cifar-10-batches-py/data_batch_4')
pf5 = pd.read_pickle('./data/cifar-10-batches-py/data_batch_5')

In [4]:
del pf1['batch_label']
del pf2['batch_label']
del pf3['batch_label']
del pf4['batch_label']
del pf5['batch_label']

In [5]:
pf1['data'] = list(pf1['data'])
pf2['data'] = list(pf2['data'])
pf3['data'] = list(pf3['data'])
pf4['data'] = list(pf4['data'])
pf5['data'] = list(pf5['data'])

In [6]:
pf1_df = pd.DataFrame(pf1)
pf2_df = pd.DataFrame(pf2)
pf3_df = pd.DataFrame(pf3)
pf4_df = pd.DataFrame(pf4)
pf5_df = pd.DataFrame(pf5)

In [7]:
train_df = pd.concat([pf1_df,
                      pf2_df,
                      pf3_df,
                      pf4_df,
                      pf5_df], ignore_index=True)

**Test dataset**

In [8]:
pf = pd.read_pickle('./data/cifar-10-batches-py/test_batch')

In [9]:
del pf['batch_label']

In [10]:
pf['data'] = list(pf['data'])

In [11]:
test_df = pd.DataFrame(pf)

<br>

## **Step 3: Transform to X_train and y_train in tensor form**

In [12]:
X_train = train_df['data'].tolist()

In [13]:
X_train = np.array(X_train)

In [14]:
X_train = torch.tensor(X_train, dtype=torch.float32).reshape((-1,3,32,32))

In [15]:
y_train = train_df['labels'].tolist()

In [16]:
y_train = np.array(y_train)

In [17]:
y_train = torch.tensor(y_train, dtype=torch.int64)

<br>

## **Step 4: Transform to X_test and y_test in tensor form**

In [18]:
X_test = test_df['data'].tolist()

In [19]:
X_test = np.array(X_test)

In [20]:
X_test = torch.tensor(X_test, dtype=torch.float32).reshape((-1,3,32,32))

In [21]:
y_test = test_df['labels'].tolist()

In [22]:
y_test = np.array(y_test)

In [23]:
y_test = torch.tensor(y_test, dtype=torch.int64)

<br>

## **Step 5: Push X_train, y_train, X_test, and y_test to GPU**

In [24]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [25]:
X_train = X_train.to(device)

In [26]:
y_train = y_train.to(device)

In [27]:
X_test = X_test.to(device)

In [28]:
y_test = y_test.to(device)

<br>

## **Step 6: Set up custom dataset (train and test) using torch.utils.data.Dataset**

In [29]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.y = y
        self.X = X
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        y = self.y[idx]
        X = self.X[idx]
        return X, y

In [30]:
train_dataset = CustomDataset(X_train, y_train)

In [31]:
test_dataset = CustomDataset(X_test, y_test)

<br>

## **Step 7: Set up custom dataloader (train and test) using torch.utils.data.DataLoader**

In [32]:
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=False)

In [33]:
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)

<br>

## **Step 8: Set up Convolutional Neural Network (CNN) model**

In [34]:
class cnn__model(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=conv2d_out, kernel_size=(3,3), padding=1, stride=1)
        self.conv2 = nn.Conv2d(in_channels=conv2d_out, out_channels=2*conv2d_out, kernel_size=(3,3), padding=1, stride=1)
        self.pool = nn.MaxPool2d(kernel_size=(2,2))
        self.fc1 = nn.Linear(2*conv2d_out*16*16, fc_out)
        self.fc2 = nn.Linear(fc_out, 10)

    def forward(self, x):
        x = nnf.relu(self.conv1(x))
        x = self.pool(nnf.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = nnf.relu(self.fc1(x))
        x = self.fc2(x)
        return x

<br>

## **Step 9: Set up test function for CNN model**

In [35]:
def test_cnn_model():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_dataloader:
            images, labels = data
            outputs = cnn_model(images)
            _, predicted = torch.max(outputs.data, 1)   # class with highest energy choosen as prediction
            total = total + labels.size(0)
            correct = correct + (predicted == labels).sum().item()
    print('correct: ', correct, '\t\t', 'total: ', total, '\t\t', 'accuracy: ', round(100*correct/total,2), '%')
    print()
    return 100*correct/total

<br>

## **Step 10: Train and test CNN model**

In [36]:
seed_value_list = []
conv2d_out_value_list = []
fc_out_value_list = []
training_duration_second_list = []
accuracy_list = []

In [41]:
seed_value = 6   # use 2, 3, 4, 5, 6

for conv2d_out_value in [6,15]:
    for fc_out_value in [20,50]:

        seed_value_list = seed_value_list + [seed_value]
        conv2d_out_value_list = conv2d_out_value_list + [conv2d_out_value]
        fc_out_value_list = fc_out_value_list + [fc_out_value]

        torch.manual_seed(seed_value)
        conv2d_out = conv2d_out_value
        fc_out = fc_out_value
        cnn_model = cnn__model().to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(cnn_model.parameters(), lr=0.0001, momentum=0.75)

        print('seed: ', seed_value, '   conv2d: ', conv2d_out_value, '   fc_out: ', fc_out_value)

        time_start = datetime.now()

        for epoch in range(50):
            running_loss = 0.0
            for i, data in enumerate(train_dataloader):
                inputs, labels = data
                optimizer.zero_grad()
                outputs = cnn_model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss = running_loss + loss.item()

        time_end = datetime.now()

        print(f"Epoch {epoch + 1}, Loss: {running_loss}")

        training_duration_second_list = training_duration_second_list + [(time_end-time_start).total_seconds()]
        print('epoch duration :', (time_end-time_start).total_seconds())

        accuracy_list = accuracy_list + [test_cnn_model()]

print('Finished Training')

seed:  6    conv2d:  6    fc_out:  20
Epoch 50, Loss: 844.2432358264923
epoch duration : 70.238213
correct:  1647 		 total:  10000 		 accuracy:  16.47 %

seed:  6    conv2d:  6    fc_out:  50
Epoch 50, Loss: 369.7464322447777
epoch duration : 70.937807
correct:  6054 		 total:  10000 		 accuracy:  60.54 %

seed:  6    conv2d:  15    fc_out:  20
Epoch 50, Loss: 334.84444880485535
epoch duration : 95.678026
correct:  6118 		 total:  10000 		 accuracy:  61.18 %

seed:  6    conv2d:  15    fc_out:  50
Epoch 50, Loss: 253.30974259972572
epoch duration : 98.183927
correct:  6289 		 total:  10000 		 accuracy:  62.89 %

Finished Training


In [42]:
df = pd.DataFrame(data=zip(seed_value_list,conv2d_out_value_list,fc_out_value_list,training_duration_second_list,accuracy_list),
                  columns=['seed','conv2d','fc','duration_second','accuracy'])

In [43]:
df['model'] = 'base_21'   # change when necessary

In [44]:
df.to_csv('base_21.csv', index=False)   # change when necessary